## Zapis i naprawa danych z formatu JSON do CSV.

Na początku tworze sobie synonimy dla najpopularniejszych nazw miast, ponieważ w niektórych przypadkach, 
miasta mają w sobie literówki np. `Waraszawa` albo nie obsłużyłem wystarczająco dużo przypadków przy miastach.
Następnie naprawiamy sobie lokacje oraz technologie. W przypadku technologii nie naprawam ich ponieważ mam pewność że są poprawne przez to ze były w jednym miejscu na stronie oraz po przejrzeniu ich nie znalazłem błędów.

In [7]:
# from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
# import pandas as pd
from typing import Dict, List, Set

locations_synonyms: Dict[str, List[str]] = {
    "Warszawa": ["Warsaw", "Warszawa", "Katowice; Warszawa", "Warszawa (Centrum)", "Waraszawa"],
    "Kraków": ["Krakow", "Kraków"],
    "Wrocław": ["Wroclaw", "Wrocław"],
    "Poznań": ["Poznan", "Poznań", "Pozanań"],
    "Gdańsk": ["Gdansk", "Gdańsk"],
    "Szczecin": ["Szczecin, szczecin"],
    "Łódź": ["Lodz", "Łódź"],
    "Rzeszów": ["Rzeszow", "Rzeszów"],
    "Katowice": ["Katowice", "Katowice; Warszawa"],
    "Kielce": ["Kielce"],
    "Opole": ["opole", "Opole"],
    "Sopot": ["Sopot"],
    "Olszytn": ["Olsztyn"],
}
    

# def normalize_locations(locations: List[str]) -> List[str]:
#     """fix your locations

#     Args:
#         locations (List[str]): list from your data 

#     Returns:
#         List[str]: list with appeared locations
#     """
#     normalized_locations: List[str] = list()
#     for tech in locations:
#         for key, values in locations_synonyms.items():
#             if tech in values:
#                 normalized_locations.append(key)
                
#     return normalized_locations

# # read data from json file and evaluate it to list
# with open("../../data/offers.json", "r") as file:
#     jobs_list = eval(file.read())

# df = pd.DataFrame(jobs_list)

# label_encoder = LabelEncoder()
# df["experience_code"] = label_encoder.fit_transform(df["experience"])
# df["operating_mode_code"] = label_encoder.fit_transform(df["operating_mode"])

# df["locations"] = df["locations"].apply(normalize_locations)

# mlb_tech = MultiLabelBinarizer()
# technologies_encoded = mlb_tech.fit_transform(df["technologies"])

# mlb_loc = MultiLabelBinarizer()
# locations_encoded = mlb_loc.fit_transform(df["locations"])

# df_encoded = df.join(pd.DataFrame(technologies_encoded, columns=mlb_tech.classes_))

# df_encoded = df_encoded.join(pd.DataFrame(locations_encoded, columns=mlb_loc.classes_))

# df_encoded = df_encoded.drop(columns=["technologies", "locations"])

# df_encoded.to_csv("../../data/jobs_with_encoded_technologies_and_locations.csv", index=False)

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
import pandas as pd

# read data from json file
df = pd.read_json("../../data/offers.json")

def normalize_location(location):
    for key, values in locations_synonyms.items():
        if location in values:
            return key
    return None

# normalizacja lokalizacji
df["locations"] = df["locations"].apply(lambda x: [normalize_location(loc) for loc in x])

# lista wszystkich lokalizacji
all_locations = list(set([loc for sublist in df["locations"].tolist() for loc in sublist]))

# utwórz LabelEncoder dla lokalizacji
location_encoder = LabelEncoder()
location_encoder.fit(all_locations)

# zakoduj lokalizacje dla każdej oferty pracy
df["location_code"] = df["locations"].apply(lambda x: [location_encoder.transform([loc])[0] for loc in x])

# utwórz nowe wiersze dla każdej lokalizacji
new_rows = []
for index, row in df.iterrows():
    for loc_code in row["location_code"]:
        new_row = row.copy()
        new_row["location_code"] = loc_code
        new_rows.append(new_row)

# utwórz nowy dataframe z nowymi wierszami
df_new = pd.DataFrame(new_rows)

# usuń kolumny 'locations' z wieloma lokalizacjami
df_new = df_new.drop(columns=["locations"])

# kodowanie LabelEncoder dla innych kategorii
label_encoder = LabelEncoder()
df_new["experience_code"] = label_encoder.fit_transform(df_new["experience"])
df_new["operating_mode_code"] = label_encoder.fit_transform(df_new["operating_mode"])

# kodowanie MultiLabelBinarizer dla technologii
mlb_tech = MultiLabelBinarizer()
technologies_encoded = mlb_tech.fit_transform(df_new["technologies"])

# dołączanie zakodowanych technologii do dataframe
df_encoded = df_new.join(pd.DataFrame(technologies_encoded, columns=mlb_tech.classes_))

df_encoded = df_encoded.drop(columns=["technologies"])

# zapisz do pliku CSV
df_encoded.to_csv("../../data/jobs_with_encoded_technologies_and_single_location.csv", index=False)


## Usuwanie niepotrzebnych danych

Na wszelki wypadek usuwam niepotrzebne dane, które mogą być niepotrzebne w przyszłości.
<p style="color: red">W przypadku ofert pracy gdzie nie ma definiowanych widełek dla np. b2b nie będę ich próbował wyliczać, bo nie ma to sensu, ponieważ zarobki zależą od technologi i poziomu doświadczenia.</p> 


In [8]:
import pandas as pd

offers = pd.read_csv("../../data/jobs_with_encoded_technologies_and_single_location.csv")

offers = offers.drop_duplicates()
offers = offers.dropna()

offers["min_b2b"] = offers["min_b2b"].apply(lambda x: x * 20 * 8 if x < 700 else x)
offers["max_b2b"] = offers["max_b2b"].apply(lambda x: x * 20 * 8 if x < 1000 else x)

offers.columns = offers.columns.str.strip()

offers.to_csv("../../data/jobs1.csv", index=False)



## Podsumowanie tego etapu

Na tym etapie mamy już czyste dane, które możemy zacząć analizować. Dokonałem normalizacji lokalizacji oraz upewniłem się, że nie ma duplikatów w ofertach albo wartości NaN (not a number). Następnie konwerujemy pensje 
go godzinowe na miesięczne oraz usuwamy spacje w kolumnach.

Można się jeszcze zastanowić nad tym czy warto podzielić dane na 3 kategorie, które dotyczą widełek (zawiera widełki):
- B2B 
- UOP
- Oba 